In [1]:
# import dependencies
import pandas as pd
import numpy as np

In [2]:
# import location data
location_df=pd.read_excel("./00_data/WPP2019_F01_LOCATIONS.XLSX",sheet_name="DB")
location_df.head()

,Index,Location,Notes,LocID,ISO3_Code,LocType,LocTypeName,ParentID,WorldID,SubRegID,...,WB_LMIC,WB_LIC,WB_NoIncomeGroup,MaxHIV_Male,MaxHIV_Female,MaxHIV_BothSexes,YearMaxHIV_BothSexes,HIVAIDSMortalityImpact_AgePattern,HIVAIDSMortalityImpact_e0,TotPop2019LessThan90k
0,1,WORLD,NaN,900,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,UN development groups,a,1803,NaN,25.0,Label/Separator,900,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,More developed regions,b,901,NaN,5.0,Development group,1803,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Less developed regions,c,902,NaN,5.0,Development group,1803,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Least developed countries,d,941,NaN,5.0,Development group,902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# extract rows that denote individual countries as a separate dataframe
country_df=location_df[location_df["LocTypeName"]=="Country/Area"]
country_df.columns

Index(['Index', 'Location', 'Notes', 'LocID', 'ISO3_Code', 'LocType',
       'LocTypeName', 'ParentID', 'WorldID', 'SubRegID', 'SubRegName',
       'SDGSubRegID', 'SDGSubRegName', 'SDGRegID', 'SDGRegName', 'GeoRegID',
       'GeoRegName', 'MoreDev', 'LessDev', 'LeastDev', 'oLessDev',
       'LessDev_ExcludingChina', 'LLDC', 'SIDS', 'WB_HIC', 'WB_MIC', 'WB_UMIC',
       'WB_LMIC', 'WB_LIC', 'WB_NoIncomeGroup', 'MaxHIV_Male', 'MaxHIV_Female',
       'MaxHIV_BothSexes', 'YearMaxHIV_BothSexes',
       'HIVAIDSMortalityImpact_AgePattern', 'HIVAIDSMortalityImpact_e0',
       'TotPop2019LessThan90k'],
      dtype='object')

In [4]:
country_df=country_df[["LocID","ISO3_Code","Location","SDGRegID","SubRegID","GeoRegID","MoreDev","LeastDev","oLessDev","WB_HIC","WB_UMIC","WB_LMIC","WB_LIC","WB_NoIncomeGroup"]]

# map country categories (i.e. which geographical region, income group etc. each country belongs to) to the country_df
# ie. here countries can belong to one of three UN development group (more developed, less developed, or least developed)
# rather than having 3 columns (MoreDev,LeastDev,and oLessDev), mapping them into a single categorical column
country_df["un_devgrp"]=None
for ind,row in country_df.iterrows():
    try:
        country_df.loc[ind,"un_devgrp"]=int(row["MoreDev"])
    except:
        try:
            country_df.loc[ind,"un_devgrp"]=int(row["LeastDev"])
        except:
            try:
                country_df.loc[ind,"un_devgrp"]=int(row["oLessDev"])
            except:
                print(row["Location"]) # ensure no row has missing devgrp

# same for world bank income group
country_df["worldbank_incgrp"]=None
for ind,row in country_df.iterrows():
    try:
        country_df.loc[ind,"worldbank_incgrp"]=int(row["WB_HIC"])
    except:
        try:
            country_df.loc[ind,"worldbank_incgrp"]=int(row["WB_UMIC"])
        except:
            try:
                country_df.loc[ind,"worldbank_incgrp"]=int(row["WB_LMIC"])
            except:
                try:
                    country_df.loc[ind,"worldbank_incgrp"]=int(row["WB_LIC"])
                except:
                    try:
                        country_df.loc[ind,"worldbank_incgrp"]=int(row["WB_NoIncomeGroup"])
                    except:
                        print(row["Location"])

# remove irrelevant columns and rename columns                     
country_df=country_df[["LocID","ISO3_Code","Location","SDGRegID","SubRegID","GeoRegID","un_devgrp","worldbank_incgrp"]]
country_df.columns=["id","iso3_code","country","sdg_region_id","subregion_id","geography_id","un_developmentgroup_id","worldbank_incomegroup_id"]
country_df

,id,iso3_code,country,sdg_region_id,subregion_id,geography_id,un_developmentgroup_id,worldbank_incomegroup_id
26,108,BDI,Burundi,947.0,910.0,903.0,941,1500
27,174,COM,Comoros,947.0,910.0,903.0,941,1500
28,262,DJI,Djibouti,947.0,910.0,903.0,941,1501
29,232,ERI,Eritrea,947.0,910.0,903.0,941,1500
30,231,ETH,Ethiopia,947.0,910.0,903.0,941,1500
...,...,...,...,...,...,...,...,...
284,60,BMU,Bermuda,1829.0,NaN,905.0,901,1503
285,124,CAN,Canada,1829.0,NaN,905.0,901,1503
286,304,GRL,Greenland,1829.0,NaN,905.0,901,1503
287,666,SPM,Saint Pierre and Miquelon,1829.0,NaN,905.0,901,1518


In [5]:
# clean missing data, convert data type
print(country_df.count())
for ind,row in country_df.iterrows():
    if np.isnan(row["subregion_id"]):
        print(location_df.loc[ind,"ParentID"])
        country_df.loc[ind,"subregion_id"]=location_df.loc[ind,"ParentID"]
country_df["sdg_region_id"]=country_df["sdg_region_id"].astype(int)
country_df["subregion_id"]=country_df["subregion_id"].astype(int)
country_df["geography_id"]=country_df["geography_id"].astype(int)
# reset index
country_df=country_df.reset_index(drop=True)

id                          235
iso3_code                   235
country                     235
sdg_region_id               235
subregion_id                230
geography_id                235
un_developmentgroup_id      235
worldbank_incomegroup_id    235
dtype: int64
918
918
918
918
918


In [6]:
# ensure no duplicate countries
duplicateindex=country_df.duplicated(subset=["id"],keep="first")
for i in duplicateindex:
    if i==True:
        print(i)

In [7]:
# save country dataframe
country_df.to_csv("./01_transform_data/country.csv",index=False)

In [8]:
# function to extract labels for different country categories
# ie. in the original location file, each country is given a category code, 
# and each category code is named in a separate row
# using this function to extract category code and name
def createCatDf(col,df):
    df_id=[]
    df_name=[]
    for i in country_df[col].unique():
        df_id.append(i)
        for ind,row in location_df.iterrows():
            if location_df.loc[ind,"LocID"]==i:
                df_name.append(location_df.loc[ind,"Location"])
    df=pd.DataFrame({
    "id":df_id,
    "name":df_name
    })
    return df

In [9]:
# function to capitalise only first letter of a country/region
def capitalise(df,col):
    for ind,row in df.iterrows():
        df.loc[ind,col]=df.loc[ind,col].title()
    return df

In [10]:
# create and save a dataframe of Sustainable Development Goal (SDG) regions
sdg_region_df=pd.DataFrame()
sdg_region_df=createCatDf("sdg_region_id",sdg_region_df)
sdg_region_df=capitalise(sdg_region_df,"name")
sdg_region_df.to_csv("./01_transform_data/sdg_region.csv",index=False)

In [11]:
# create and save a dataframe of subregions
subregion_df=pd.DataFrame()
subregion_df=createCatDf("subregion_id",subregion_df)
subregion_df=capitalise(subregion_df,"name")
subregion_df.to_csv("./01_transform_data/subregion.csv",index=False)

In [12]:
# create and save a dataframe of geographical regions
geog_region_df=pd.DataFrame()
geog_region_df=createCatDf("geography_id",geog_region_df)
geog_region_df=capitalise(geog_region_df,"name")
geog_region_df.to_csv("./01_transform_data/geog_region.csv",index=False)

In [13]:
# create and save a dataframe of UN development group regions
un_devgrp_df=pd.DataFrame()
un_devgrp_df=createCatDf("un_developmentgroup_id",un_devgrp_df)
un_devgrp_df=capitalise(un_devgrp_df,"name")
un_devgrp_df.to_csv("./01_transform_data/un_devgrp.csv",index=False)

In [14]:
# create and save a dataframe of World Bank income group regions
wb_incgrp_df=pd.DataFrame()
wb_incgrp_df=createCatDf("worldbank_incomegroup_id",wb_incgrp_df)
wb_incgrp_df=capitalise(wb_incgrp_df,"name")
wb_incgrp_df.to_csv("./01_transform_data/worldbank_incgrp.csv",index=False)

In [5]:
# import population data
population_df=pd.read_csv("./00_data/WPP2019_PopulationByAgeSex_Medium.csv")
population_df.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0-4,0,5,630.044,661.578,1291.622
1,4,Afghanistan,2,Medium,1950,1950.5,5-9,5,5,516.206,487.335,1003.541
2,4,Afghanistan,2,Medium,1950,1950.5,10-14,10,5,461.378,423.326,884.704
3,4,Afghanistan,2,Medium,1950,1950.5,15-19,15,5,414.369,369.363,783.732
4,4,Afghanistan,2,Medium,1950,1950.5,20-24,20,5,374.109,318.392,692.501


In [16]:
# clean population data
population_df["AgeGrpEnd"]=population_df["AgeGrpStart"]+4
population_df=population_df[["LocID","Time","AgeGrpStart","AgeGrpEnd","PopMale","PopFemale","PopTotal"]]
# remove projections into the future (population survey conducted in 2019)
population_df=population_df[population_df["Time"]<=2019]
population_df.head()

,LocID,Time,AgeGrpStart,AgeGrpEnd,PopMale,PopFemale,PopTotal
0,4,1950,0,4,630.044,661.578,1291.622
1,4,1950,5,9,516.206,487.335,1003.541
2,4,1950,10,14,461.378,423.326,884.704
3,4,1950,15,19,414.369,369.363,783.732
4,4,1950,20,24,374.109,318.392,692.501


In [17]:
# match country id across country_df and population_df to ensure no unrepresented country codes
country_id=country_df.id.tolist()
population_df=population_df[population_df["LocID"].isin(country_id)]
# ensure no duplicate rows in population_df
duplicateindex=population_df.duplicated(subset=["LocID","Time","AgeGrpStart"],keep="first")
for i in duplicateindex:
    if i==True:
        print(i)

In [18]:
# rename columns and save population_df
population_df.columns=["country_id","year","age_start","age_end","population_male_thousands","population_female_thousands","population_total_thousands"]
population_df=population_df.reset_index(drop=True)
population_df.to_csv("./01_transform_data/population.csv",index=False)

In [19]:
# import demographic data
demographic_df=pd.read_excel("./00_data/WPP2019_INT_F01_ANNUAL_DEMOGRAPHIC_INDICATORS.xlsx",header=16)
demographic_df.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,Reference date (1 January - 31 December),Deaths (thousands),Male deaths (thousands),...,"Infant deaths, both sexes combined (under age 1, thousands)","Infant mortality rate (infant deaths per 1,000 live births)","Under-five mortality (deaths under age 5 per 1,000 live births)",Births (thousands),"Crude birth rate (births per 1,000 population)",Total fertility (live births per woman),Total population natural change / increase (thousands),"Rate of natural increase (per 1,000 population)",Population change (thousands),Population growth rate (percentage)
0,1,Estimates,WORLD,NaN,900,World,0,1950,51344.8,26761,...,14213,145.904,223.391,97375.3,37.915,5.05,46030.5,17.928,47162.7,1.859
1,2,Estimates,WORLD,NaN,900,World,0,1951,51137.3,26667.5,...,14053.2,144.157,220.441,97429.8,37.634,5.024,46292.5,17.886,47220.9,1.828
2,3,Estimates,WORLD,NaN,900,World,0,1952,50778.7,26507,...,13764,140.826,214.83,97655,37.11,4.977,46876.3,17.818,46789.6,1.779
3,4,Estimates,WORLD,NaN,900,World,0,1953,50533.1,26399.3,...,13535.3,137.826,209.797,98112.8,36.657,4.941,47579.7,17.782,46992.2,1.755
4,5,Estimates,WORLD,NaN,900,World,0,1954,50400.4,26344.5,...,13367.2,135.156,205.342,98803.2,36.275,4.916,48402.9,17.778,47703.2,1.751


In [20]:
# remove projections into the future
demographic_df=demographic_df[demographic_df["Reference date (1 January - 31 December)"]<=2019]
# ensure no unrepresented countries in the country_df
demographic_df=demographic_df[demographic_df["Type"]=="Country/Area"]
for i in demographic_df["Country code"]:
    if i not in country_id:
        print(i)

In [21]:
demographic_df.columns

Index(['Index', 'Variant', 'Region, subregion, country or area *', 'Notes',
       'Country code', 'Type', 'Parent code',
       'Reference date (1 January - 31 December)', 'Deaths (thousands)',
       'Male deaths (thousands)', 'Female deaths (thousands)',
       'Crude death rate (deaths per 1,000 population)',
       'Life expectancy at birth, both sexes combined (years)',
       'Life expectancy at birth, males (years)',
       'Life expectancy at birth, females (years)',
       'Infant deaths, both sexes combined (under age 1, thousands)',
       'Infant mortality rate (infant deaths per 1,000 live births)',
       'Under-five mortality (deaths under age 5 per 1,000 live births)',
       'Births (thousands)', 'Crude birth rate (births per 1,000 population)',
       'Total fertility (live births per woman)',
       'Total population natural change / increase (thousands)',
       'Rate of natural increase (per 1,000 population)',
       'Population change (thousands)', 'Population g

In [22]:
# rename columns
demographic_df.columns=["id","variant","country","notes","country_id","type","parent_code","year","death_thousands","death_male_thousands","death_female_thousands","crude_death","life_exp","life_exp_male","life_exp_female","infant_death","infant_mortality","underfive_mortality","birth_thousands","crude_birth","total_fertility","total_pop_natural_change","rate_natural_increase","pop_change","pop_growth_percent"]
demographic_df=demographic_df[["country_id","year","death_thousands","death_male_thousands","death_female_thousands","crude_death","life_exp","life_exp_male","life_exp_female","infant_death","infant_mortality","underfive_mortality","birth_thousands","crude_birth","total_fertility","total_pop_natural_change","rate_natural_increase","pop_change","pop_growth_percent"]]
demographic_df.head()

,country_id,year,death_thousands,death_male_thousands,death_female_thousands,crude_death,life_exp,life_exp_male,life_exp_female,infant_death,infant_mortality,underfive_mortality,birth_thousands,crude_birth,total_fertility,total_pop_natural_change,rate_natural_increase,pop_change,pop_growth_percent
1846,108,1950,60.47,30.916,29.554,25.929,38.422,36.856,39.987,20.154,169.825,286.357,118.338,50.695,6.814,57.868,24.766,50.95,2.207
1847,108,1951,60.719,31.058,29.662,25.787,38.574,37.008,40.138,20.15,168.932,284.86,118.957,50.494,6.807,58.238,24.706,48.587,2.06
1848,108,1952,61.215,31.337,29.878,25.505,38.877,37.311,40.44,20.151,167.148,281.873,120.265,50.113,6.798,59.05,24.608,44.337,1.843
1849,108,1953,61.706,31.61,30.096,25.223,39.179,37.612,40.741,20.17,165.369,278.901,121.711,49.774,6.795,60.005,24.551,43.08,1.759
1850,108,1954,62.192,31.877,30.316,24.941,39.479,37.911,41.041,20.208,163.594,275.943,123.297,49.477,6.798,61.105,24.537,44.024,1.766


In [23]:
# ensure no duplicate rows
duplicateindex=demographic_df.duplicated(subset=["country_id","year"],keep="first")
for i in duplicateindex:
    if i==True:
        print(i)

In [24]:
# save demographic data
demographic_df.to_csv("./01_transform_data/demographic.csv",index=False)